# Visualising Eye Movement

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
import numpy as np

import matplotlib
matplotlib.use("Agg") 

In [2]:
import sys
import os

sys.path.append(os.path.abspath("../src"))  # Add folder_a to sys.path
from config import *

In [3]:
experiment = "EVIL_BASTARD"
participant_ids = pd.read_parquet(
        f"{CLEANED_DIR}/{experiment}_samples.pq", 
        columns=["participant_id"]
    )

participant_ids = participant_ids["participant_id"].unique()

In [20]:
# Read data for participant
participant_id = participant_ids[10]
filters = [
    ('participant_id', '=', participant_id),
]

samples_df = pd.read_parquet(
    f"{CLEANED_DIR}/{experiment}_samples.pq",
    filters=filters
    )

events_df = pd.read_parquet(
    f"{CLEANED_DIR}/{experiment}_events.pq",
    filters=filters
    )

In [21]:
# Extract fixpoints
fixpoints = events_df[events_df["event"]=="FIXPOINT"].loc[:,["trial_id", "time", "event", "stimulus_x", "stimulus_y"]]

In [22]:
trial_id=0

sample_df = samples_df.loc[samples_df["trial_id"]==trial_id,:]

# Extract fixpoints
event_df = events_df.loc[events_df["trial_id"]==trial_id,:]
fixpoints_df = event_df[event_df["event"]=="FIXPOINT"].loc[:,["trial_id", "time", "event", "colour", "stimulus_x", "stimulus_y"]]

# Insert fixpoints in sample data
# Ensure you are modifying actual copies
sample_df = sample_df.copy()
fixpoints_df = fixpoints_df.copy()

# Force types
sample_df["time"] = sample_df["time"].astype("float64")
fixpoints_df["time"] = fixpoints_df["time"].astype("float64")

# Make sure both DataFrames are sorted by time
sample_df = sample_df.sort_values("time")
fixpoints_df = fixpoints_df.sort_values("time")

# Rename 'colour' column to 'fixpoint' so it's ready to merge
fixpoints_df = fixpoints_df.rename(columns={"colour": "fixpoint"})

# Perform a backward-looking join: for each row in sample_df, find the most recent fixpoint time
sample_df = pd.merge_asof(
    sample_df,
    fixpoints_df,
    on="time",
    direction="backward"
)

sample_df["fixpoint"] = sample_df["fixpoint"].map({RED:"red", GREEN:"green", BLUE:"blue", WHITE:"white"})

In [23]:
x = sample_df.copy()
x_sampled = x[x.index % 100 == 0]

# Apply Seaborn style
sns.set(style="whitegrid")

fig, ax = plt.subplots(1, 1, figsize = (6, 6))

# Initialize the scatter plot object
scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color
scatter_fixpoint = ax.scatter([], [], label="Fixpoint", alpha=0.6, s=50)  # 's' is for size, 'c' is for color

x_min, x_max = 0, 1920 
y_min, y_max = 0, 1080 
ax.set_xlim([x_min, x_max]) 
ax.set_ylim([y_min, y_max]) 

# Add legend to distinguish between the three points
ax.legend()

def animate(i):
    # Update the scatter plot data for each frame
    
    # Update coordinates
    coords_right = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_right"].iloc[i]]
    coords_left = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_left"].iloc[i]] 
    coords_fixpoint = np.c_[x_sampled["stimulus_x"].iloc[i], x_sampled["stimulus_y"].iloc[i]] 

    scatter_right.set_offsets(coords_right)    
    scatter_left.set_offsets(coords_left)
    scatter_fixpoint.set_offsets(coords_fixpoint)

    # Update colour based on fixpoint
    fixpoint_color = x_sampled["fixpoint"].iloc[i]
    if pd.isna(fixpoint_color):
        fixpoint_color = "gray"
    elif fixpoint_color == "white":
        fixpoint_color = "black"
    scatter_fixpoint.set_color([fixpoint_color])

    return scatter_right, scatter_left, scatter_fixpoint

anim = animation.FuncAnimation(
    fig, 
    animate, 
    frames = len(x_sampled), 
    interval=100,
    blit = False
)

anim.save(f"Animations/{experiment}_{participant_id}_{trial_id}.mp4", writer="ffmpeg", fps=30)



/tmp/ipykernel_1750/917337376.py:10: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
/tmp/ipykernel_1750/917337376.py:11: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color


In [24]:
sample_df

,experiment,participant_id,trial_id_x,time,x_left,y_left,pupil_size_left,x_velocity_left,y_velocity_left,x_right,...,x_velocity_right,y_velocity_right,x_resolution,y_resolution,error_message,trial_id_y,event,fixpoint,stimulus_x,stimulus_y
0,EVIL_BASTARD,138,0,4488462.0,893.5,498.2,1839.0,NaN,NaN,952.9,...,NaN,NaN,54.5,55.6,.....,NaN,NaN,NaN,NaN,NaN
1,EVIL_BASTARD,138,0,4488462.0,893.5,498.0,1840.0,NaN,NaN,952.6,...,NaN,NaN,54.5,55.6,.....,NaN,NaN,NaN,NaN,NaN
2,EVIL_BASTARD,138,0,4488463.0,893.0,497.8,1840.0,NaN,NaN,953.4,...,NaN,NaN,54.5,55.6,.....,NaN,NaN,NaN,NaN,NaN
3,EVIL_BASTARD,138,0,4488463.0,892.1,498.6,1835.0,NaN,NaN,954.3,...,NaN,NaN,54.5,55.6,.....,NaN,NaN,NaN,NaN,NaN
4,EVIL_BASTARD,138,0,4488464.0,893.1,500.4,1831.0,NaN,NaN,955.2,...,NaN,NaN,54.5,55.6,.....,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24380,EVIL_BASTARD,138,0,4500652.0,1285.8,518.3,954.0,NaN,NaN,1217.0,...,NaN,NaN,54.9,55.7,.....,0.0,FIXPOINT,red,1241.0,475.0
24381,EVIL_BASTARD,138,0,4500652.0,1286.4,518.2,955.0,NaN,NaN,1217.5,...,NaN,NaN,54.9,55.7,.....,0.0,FIXPOINT,red,1241.0,475.0
24382,EVIL_BASTARD,138,0,4500653.0,1287.0,516.8,953.0,NaN,NaN,1218.1,...,NaN,NaN,54.9,55.7,.....,0.0,FIXPOINT,red,1241.0,475.0
24383,EVIL_BASTARD,138,0,4500653.0,1287.0,517.4,956.0,NaN,NaN,1218.1,...,NaN,NaN,54.9,55.7,.....,0.0,FIXPOINT,red,1241.0,475.0
